# How to calculate Elterngeld

Elterngeld (parental leave benefit) is a financial support for parents in the first
months after the birth of a child. It replaces some share of the income before birth
(usually about 67% of net income). This net income is approximated using simplified
calculation rules. This document explains how to calculate Elterngeld
based on historical income data for a given household.

In principle, one can compute Elterngeld in three steps:
1. Compute the average monthly gross income before birth in the data.
2. Call GETTSIM with the target `elterngeld_nettolohn_approximation_m`.
3. Call GETTSIM with the target `elterngeld_m` using the outcome of step 2 as the input
   for `elterngeld_nettoeinkommen_vorjahr_m`.

In the following, we will explain some more details.

## Example

In the following, we compute Elterngeld for a child that is born in July 2024. The
parent that receives Elterngeld had a monthly gross wage of 2000€ in the 12 months
before birth of the child.

### Step 1: Compute the average monthly gross income before birth

If income from dependent employment is the only source of income, the basis for the
Elterngeld calculation is the average monthly gross income in the 12 months before the
birth of a child. In our case, this would be July 2023 until June 2024. If other income
sources exist, the basis is the average monthly gross income in the calendar year before
the birth of the child, 2023 in our example. We will stick to the first case here
because:

> **Note**: The Elterngeld implementation in GETTSIM does currently not consider income
> from self-employment or other sources. See
> [this](https://github.com/iza-institute-of-labor-economics/gettsim/issues/613) issue.

So let's stick to the case where income from dependent employment is the only source of
income. You can compute this directly if you have monthly information on gross income
(e.g. IAB or DRV data). If you have only annual income data, you must choose an
approximation. In most cases, a good approximation is the income of the year before
birth.

Here, we use made-up data using the `create_synthetic_data` function.


In [ ]:
from gettsim import (
    compute_taxes_and_transfers,
    create_synthetic_data,
    set_up_policy_environment,
)

In [ ]:
params, functions = set_up_policy_environment(2024)

data_before_birth = create_synthetic_data(
    n_adults=2,
    n_children=0,
    specs_heterogeneous={
        "bruttolohn_m": [[2000.0, 0.0]],
    },
)

### Step 2: Approximate net wage before birth

GETTSIM provides an easy way to compute the relevant net wage
`elterngeld_nettoeinkommen_vorjahr_m` based on step 1 using the target `elterngeld_nettolohn_approximation_m`.

In [ ]:
net_wage_approximation = compute_taxes_and_transfers(
    data=data_before_birth,
    functions=functions,
    params=params,
    targets=["elterngeld_nettolohn_approximation_m"],
)

net_wage_approximation

### Step 3: Compute Elterngeld

Finally, we can compute Elterngeld using the target `elterngeld_m` and the net wage
information from step 2.

The reason for not doing this in one GETTSIM call (together with step 2) is that 
1. the correct policy date of the net wage calculation (step 2) is always January 1st of
the year the child was born (disregarding potential parameter changes throughout the
year). If there was no parameter change (or you're fine with neglecting this), you can
choose the same policy date as in step 2.
2. other state variables of the household (e.g. place of residence) might change
after birth of the child.

First, we create the new data set after birth.

In [ ]:
# Create data
data_after_birth = create_synthetic_data(
    n_adults=2,
    n_children=1,
    specs_heterogeneous={
        "bruttolohn_m": [[0.0, 0.0, 0.0]],
        "geburtsjahr": [[1980, 1980, 2024]],
        "geburtsmonat": [[1, 1, 7]],
        "alter": [[44, 44, 0]],
        "elterngeld_claimed": [[True, False, False]],  # First adult claims Elterngeld
    },
)

Then, we add `elterngeld_nettoeinkommen_vorjahr_m` to the data based on step 2.

In [ ]:
# Add net wage approximation
data_after_birth["elterngeld_nettoeinkommen_vorjahr_m"] = net_wage_approximation[
    "elterngeld_nettolohn_approximation_m"
]

Then, we compute Elterngeld.

In [ ]:
# Compute Elterngeld
results = compute_taxes_and_transfers(
    data=data_after_birth,
    functions=functions,
    params=params,
    targets=["elterngeld_m"],
)

results